# Search Analytics mit DuckDB

Dieses Notebook hilft dir bei der Analyse deiner Search Analytics Daten mit DuckDB.

---
# QUICK START

**Nur diese 2 Zellen ausführen, um loszulegen!**

### Schritt 1: Passe den Pfad zu deiner CSV-Datei an

In [ ]:
#################################################################
#  EINZIGE EINSTELLUNG: Pfad zu deiner CSV-Datei               #
#################################################################

CSV_PATH = '../data/search_export.csv'    # <-- HIER ANPASSEN!

#################################################################

### Schritt 2: Diese Zelle ausführen - alles andere passiert automatisch

In [ ]:
# ===== AUTOMATISCHES SETUP =====
# Diese Zelle:
# 1. Importiert alle benötigten Libraries
# 2. Erstellt die Datenbank (.db Datei)
# 3. Liest deine CSV und erstellt automatisch eine Tabelle
# 4. Zeigt dir was importiert wurde

import duckdb
import pandas as pd
from pathlib import Path

# Plotting optional
try:
    import matplotlib.pyplot as plt
    import matplotlib.dates as mdates
    plt.style.use('seaborn-v0_8-whitegrid')
    PLOTTING_AVAILABLE = True
except ImportError:
    PLOTTING_AVAILABLE = False

# Pandas Display-Optionen
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Datenbank erstellen (im gleichen Ordner wie die CSV)
csv_path = Path(CSV_PATH)
db_path = csv_path.parent / 'searchanalytics.db'

# Verbindung herstellen
con = duckdb.connect(str(db_path))

# Hilfsfunktionen
def query(sql):
    """SQL ausführen und DataFrame zurückgeben"""
    return con.execute(sql).df()

def execute(sql):
    """SQL ausführen ohne Rückgabe"""
    con.execute(sql)

# Tabelle erstellen aus CSV (Schema wird automatisch erkannt!)
print("="*60)
print("AUTOMATISCHER IMPORT")
print("="*60)

# Falls Tabelle bereits existiert, löschen und neu erstellen
execute("DROP TABLE IF EXISTS searches")

# CSV einlesen - DuckDB erkennt automatisch:
# - Spaltentypen (TEXT, INTEGER, TIMESTAMP, etc.)
# - Trennzeichen (Komma, Semikolon, Tab)
# - Header (erste Zeile als Spaltennamen)
execute(f"""
    CREATE TABLE searches AS
    SELECT * FROM read_csv('{CSV_PATH}', auto_detect=true)
""")

# Was wurde importiert?
row_count = query("SELECT COUNT(*) as n FROM searches")['n'][0]
print(f"\n CSV-Datei: {csv_path.name}")
print(f" Datenbank: {db_path.name}")
print(f" Importiert: {row_count:,} Zeilen")

print("\n" + "="*60)
print("ERKANNTE SPALTEN")
print("="*60)
schema = query("DESCRIBE searches")
for _, row in schema.iterrows():
    print(f"  {row['column_name']:30} {row['column_type']}")

print("\n" + "="*60)
print("ERSTE 5 ZEILEN")
print("="*60)
display(query("SELECT * FROM searches LIMIT 5"))

print("\n Setup abgeschlossen! Du kannst jetzt die Analyse-Zellen unten ausführen.")

---
# ANALYSEN

Ab hier kannst du die Zellen ausführen, die dich interessieren.

**Wichtig:** Falls deine Spalten anders heißen, passe die Namen in den Queries an!
Typische Spalten-Varianten:
- Zeitstempel: `timestamp`, `date`, `datetime`, `created_at`
- Suchbegriff: `search_query`, `query`, `search_term`, `keyword`
- Ergebnisse: `results_count`, `result_count`, `hits`, `total_results`
- Antwortzeit: `response_time`, `duration`, `latency_ms`

---
## Basis-Statistiken

In [ ]:
# Übersicht: Was haben wir?
query("DESCRIBE searches")

In [ ]:
# Erste und letzte Einträge
query("""
    SELECT
        COUNT(*) as total_rows,
        MIN(timestamp) as first_entry,
        MAX(timestamp) as last_entry
    FROM searches
""")

In [ ]:
# Beispieldaten anschauen
query("SELECT * FROM searches LIMIT 20")

---
## Zeitliche Verteilung

In [ ]:
# Einträge pro Tag
# HINWEIS: Ersetze 'timestamp' durch deine Datumsspalte falls nötig

query("""
    SELECT
        DATE_TRUNC('day', timestamp)::DATE as datum,
        COUNT(*) as anzahl
    FROM searches
    GROUP BY 1
    ORDER BY 1 DESC
    LIMIT 30
""")

In [ ]:
# Verteilung nach Stunde
query("""
    SELECT
        EXTRACT(HOUR FROM timestamp) as stunde,
        COUNT(*) as anzahl,
        ROUND(100.0 * COUNT(*) / SUM(COUNT(*)) OVER(), 2) as prozent
    FROM searches
    GROUP BY 1
    ORDER BY 1
""")

In [ ]:
# Verteilung nach Wochentag
query("""
    SELECT
        DAYNAME(timestamp) as wochentag,
        DAYOFWEEK(timestamp) as tag_nr,
        COUNT(*) as anzahl
    FROM searches
    GROUP BY 1, 2
    ORDER BY 2
""")

---
## Top Werte (für jede Spalte anpassbar)

In [ ]:
# Top 20 häufigste Werte einer Spalte
# HINWEIS: Ersetze 'search_query' durch die Spalte die dich interessiert

SPALTE = 'search_query'  # <-- Hier anpassen

query(f"""
    SELECT
        {SPALTE},
        COUNT(*) as anzahl,
        ROUND(100.0 * COUNT(*) / SUM(COUNT(*)) OVER(), 2) as prozent
    FROM searches
    WHERE {SPALTE} IS NOT NULL
    GROUP BY 1
    ORDER BY 2 DESC
    LIMIT 20
""")

---
## Null-Ergebnis-Analyse

Falls du eine Spalte mit Ergebnis-Anzahl hast (z.B. `results_count`)

In [ ]:
# Gesamt Null-Rate
# HINWEIS: Ersetze 'results_count' durch deine Spalte

query("""
    SELECT
        COUNT(*) as total_searches,
        SUM(CASE WHEN results_count = 0 THEN 1 ELSE 0 END) as null_results,
        ROUND(100.0 * SUM(CASE WHEN results_count = 0 THEN 1 ELSE 0 END) / COUNT(*), 2) as null_rate_pct
    FROM searches
""")

In [ ]:
# Suchbegriffe mit den meisten Null-Ergebnissen
query("""
    SELECT
        search_query,
        COUNT(*) as anzahl,
        SUM(CASE WHEN results_count = 0 THEN 1 ELSE 0 END) as null_results,
        ROUND(100.0 * SUM(CASE WHEN results_count = 0 THEN 1 ELSE 0 END) / COUNT(*), 1) as null_rate_pct
    FROM searches
    WHERE search_query IS NOT NULL
    GROUP BY 1
    HAVING COUNT(*) >= 5  -- Mindestens 5 Suchen
    ORDER BY null_results DESC
    LIMIT 20
""")

In [ ]:
# Null-Rate pro Tag
query("""
    SELECT
        DATE_TRUNC('day', timestamp)::DATE as datum,
        COUNT(*) as total,
        SUM(CASE WHEN results_count = 0 THEN 1 ELSE 0 END) as null_results,
        ROUND(100.0 * SUM(CASE WHEN results_count = 0 THEN 1 ELSE 0 END) / COUNT(*), 2) as null_rate_pct
    FROM searches
    GROUP BY 1
    ORDER BY 1 DESC
    LIMIT 30
""")

---
## Performance-Metriken

Falls du eine Spalte mit Antwortzeit hast (z.B. `response_time`)

In [ ]:
# Response Time Statistiken
# HINWEIS: Ersetze 'response_time' durch deine Spalte

query("""
    SELECT
        COUNT(*) as total,
        ROUND(AVG(response_time), 2) as avg_ms,
        ROUND(MEDIAN(response_time), 2) as median_ms,
        ROUND(PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY response_time), 2) as p90_ms,
        ROUND(PERCENTILE_CONT(0.95) WITHIN GROUP (ORDER BY response_time), 2) as p95_ms,
        ROUND(PERCENTILE_CONT(0.99) WITHIN GROUP (ORDER BY response_time), 2) as p99_ms,
        ROUND(MAX(response_time), 2) as max_ms
    FROM searches
    WHERE response_time IS NOT NULL
""")

---
## Visualisierungen

Falls Matplotlib installiert ist (`conda install matplotlib`)

In [ ]:
if PLOTTING_AVAILABLE:
    # Einträge pro Tag
    daily = query("""
        SELECT
            DATE_TRUNC('day', timestamp)::DATE as datum,
            COUNT(*) as anzahl
        FROM searches
        GROUP BY 1
        ORDER BY 1
    """)
    
    fig, ax = plt.subplots(figsize=(14, 5))
    ax.plot(daily['datum'], daily['anzahl'], linewidth=2, color='steelblue')
    ax.fill_between(daily['datum'], daily['anzahl'], alpha=0.3, color='steelblue')
    ax.set_title('Einträge pro Tag', fontsize=14, fontweight='bold')
    ax.set_xlabel('Datum')
    ax.set_ylabel('Anzahl')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
else:
    print("Matplotlib nicht installiert. Führe aus: conda install matplotlib")

In [ ]:
if PLOTTING_AVAILABLE:
    # Verteilung nach Stunde
    hourly = query("""
        SELECT
            EXTRACT(HOUR FROM timestamp)::INT as stunde,
            COUNT(*) as anzahl
        FROM searches
        GROUP BY 1
        ORDER BY 1
    """)
    
    fig, ax = plt.subplots(figsize=(12, 5))
    ax.bar(hourly['stunde'], hourly['anzahl'], color='steelblue')
    ax.set_title('Verteilung nach Tageszeit', fontsize=14, fontweight='bold')
    ax.set_xlabel('Stunde')
    ax.set_ylabel('Anzahl')
    ax.set_xticks(range(0, 24))
    plt.tight_layout()
    plt.show()

In [ ]:
if PLOTTING_AVAILABLE:
    # Top 10 Werte
    SPALTE = 'search_query'  # <-- Anpassen
    
    top = query(f"""
        SELECT {SPALTE} as wert, COUNT(*) as anzahl
        FROM searches
        WHERE {SPALTE} IS NOT NULL AND {SPALTE} != ''
        GROUP BY 1
        ORDER BY 2 DESC
        LIMIT 10
    """)
    
    fig, ax = plt.subplots(figsize=(10, 6))
    y_pos = range(len(top))
    ax.barh(y_pos, top['anzahl'], color='steelblue')
    ax.set_yticks(y_pos)
    ax.set_yticklabels(top['wert'])
    ax.invert_yaxis()
    ax.set_title(f'Top 10: {SPALTE}', fontsize=14, fontweight='bold')
    ax.set_xlabel('Anzahl')
    plt.tight_layout()
    plt.show()

---
## Eigene Queries

Hier kannst du eigene SQL-Queries schreiben:

In [ ]:
# Deine eigene Query hier:
query("""
    SELECT *
    FROM searches
    LIMIT 10
""")

In [ ]:
# Noch eine Query:
query("""
    SELECT *
    FROM searches
    LIMIT 10
""")

---
## Export

In [ ]:
# Ergebnis einer Query als CSV exportieren
result = query("""
    SELECT *
    FROM searches
    LIMIT 1000
""")

result.to_csv('../output/export.csv', index=False)
print("Exportiert: ../output/export.csv")

In [ ]:
# Komplette Tabelle als Parquet (schneller + kleiner als CSV)
execute("COPY searches TO '../output/searches.parquet' (FORMAT PARQUET)")
print("Exportiert: ../output/searches.parquet")

---
## Neue Daten hinzufügen

Falls du später weitere CSV-Dateien importieren möchtest:

In [ ]:
# Neue CSV an bestehende Tabelle anhängen
# NEW_CSV = '../data/neue_daten.csv'

# execute(f"""
#     INSERT INTO searches
#     SELECT * FROM read_csv('{NEW_CSV}', auto_detect=true)
# """)

# print(f"Neue Daten hinzugefügt. Gesamt: {query('SELECT COUNT(*) FROM searches')['count_star()'][0]:,} Zeilen")

---
## Aufräumen

In [ ]:
# Verbindung schließen (am Ende der Session)
con.close()
print("Verbindung geschlossen")

---
## Notizen

**Meine Spalten:**
- ...

**Erkenntnisse:**
- ...

**Offene Fragen:**
- ...